In [1]:
import adanet
import tensorflow as tf
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout

class CNNBuilder(adanet.subnetwork.Builder):
    def __init__(self, n_convs):
        self._n_convs = n_convs
        
    def build_subnetwork(self,
                         features,
                         logits_dimension,
                         training,
                         iteration_step,
                         summary,
                         previous_ensemble=None):
        
        images = list(features.values())[0]
        x = images
    
        for i in range(self._n_convs):
            x = Conv2D(32, kernel_size=7, activation='relu')(x)
            x = MaxPooling2D(strides=2)(x)
        
        x = Flatten()(x)
        x = Dense(100, activation='relu')(x)
        
        logits = Dense(10)(x)

        complexity = tf.constant(1)

        persisted_tensors = {'n_convs': tf.constant(self._n_convs)}
        
        return adanet.Subnetwork(
            last_layer=x,
            logits=logits,
            complexity=complexity,
            persisted_tensors=persisted_tensors)
    
    def build_subnetwork_train_op(self,
                                  subnetwork,
                                  loss,
                                  var_list,
                                  labels,
                                  iteration_step,
                                  summary,
                                  previous_ensemble=None):

        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001,
                                              decay=0.0)
        return optimizer.minimize(loss=loss, var_list=var_list)

    def build_mixture_weights_train_op(self,
                                       loss,
                                       var_list,
                                       logits,
                                       labels,
                                       iteration_step, summary):
        return tf.no_op("mixture_weights_train_op")

    @property
    def name(self):
        return f'cnn_{self._n_convs}'

Using TensorFlow backend.


In [3]:
class CNNGenerator(adanet.subnetwork.Generator):
    def __init__(self):
            self._cnn_builder_fn = CNNBuilder

    def generate_candidates(self,
                                previous_ensemble,
                                iteration_number,
                                previous_ensemble_reports,
                                all_reports):

        n_convs = 0
        if previous_ensemble:
            n_convs = tf.contrib.util.constant_value(
                previous_ensemble.weighted_subnetworks[-1]
                .subnetwork
                .persisted_tensors['n_convs'])
        return [
                    self._cnn_builder_fn(n_convs=n_convs),
                    self._cnn_builder_fn(n_convs=n_convs + 1)
                ]

In [4]:
import p5_util
import p8_util

datadir = './P7/data'
filename=datadir+'/arr_keras_X_y_train_test.dump'
(x_train,x_test, y_train, y_test) = p5_util.object_load(filename)
if True :
    y_train=p8_util.array_label_encode_from_index(y_train)
    y_test=p8_util.array_label_encode_from_index(y_test)

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

w_size = x_train.shape[1]
h_size = x_train.shape[2]        

labels_train = y_train.astype(np.int32) # cast values to int32
labels_test = y_test.astype(np.int32)   # cast values to int32

p5_util.object_load : fileName= ./P7/data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414,) (47,)


In [5]:
labels_test = labels_test.reshape(-1,1)
labels_train = labels_train.reshape(-1,1)
labels_train.shape


(414, 1)

In [6]:
labels_train[0]

array([0], dtype=int32)

In [5]:
head = tf.contrib.estimator.multi_class_head(10)
ADANET_ITERATIONS = 2  #@param {type:"integer"}
TRAIN_STEPS = 100  #@param {type:"integer"}
max_iteration_steps = int(TRAIN_STEPS / ADANET_ITERATIONS)

EPOCHS = 10
BATCH_SIZE = 32
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=labels_train,
    batch_size=BATCH_SIZE,
    num_epochs=EPOCHS,
    shuffle=False)

adanet_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=labels_train,
    batch_size=BATCH_SIZE,
    num_epochs=1,
    shuffle=False)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=labels_test,
    batch_size=BATCH_SIZE,
    num_epochs=1,
    shuffle=False)


estimator = adanet.Estimator(
    head=head,
    subnetwork_generator=CNNGenerator(),
    max_iteration_steps=max_iteration_steps,
    evaluator=adanet.Evaluator(
        input_fn=adanet_input_fn,
        steps=None),
    adanet_loss_decay=.99)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7550ry0e', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3af93d8a58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn=train_input_fn,
        max_steps=TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn=test_input_fn,
        steps=None))
print("Accuracy:", results["accuracy"])
print("Loss:", results["average_loss"])
print(ensemble_architecture(results))

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Beginning training AdaNet iteration 0
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Building iteration 0
INFO:tensorflow:Building subnetwork 'cnn_0'
Instructions for updating:
`persisted_tensors` is deprecated, please use `shared` instead.

+++ _check_logits_final_dim() : expected_logits_dimension= 10

+++ _check_logits_final_dim() : expected_logits_dimension= 10

+++ _check_logits_final_dim() : expected_logits_dimension= 10
INFO:tensorflow:Building subnetwork 'cnn_

ValueError: Tensor adanet/iteration_0/ensemble_t0_cnn_1/weighted_subnetwork_0/subnetwork/dense_1/kernel is not found in /tmp/tmp7550ry0e/model.ckpt-50 checkpoint {'current_iteration': [], 'adanet/iteration_0/step': [], 'adanet/iteration_0/ensemble_t0_cnn_1/weighted_subnetwork_0/subnetwork/dense_4/kernel': [100, 10], 'adanet/iteration_0/ensemble_t0_cnn_1/weighted_subnetwork_0/subnetwork/dense_4/bias': [10], 'adanet/iteration_0/ensemble_t0_cnn_1/weighted_subnetwork_0/subnetwork/dense_3/bias': [100], 'adanet/iteration_0/ensemble_t0_cnn_0/train_subnetwork/adanet/iteration_0/ensemble_t0_cnn_0/weighted_subnetwork_0/subnetwork/dense_2/bias/RMSProp': [10], 'adanet/iteration_0/ensemble_t0_cnn_0/train_subnetwork/adanet/iteration_0/ensemble_t0_cnn_0/weighted_subnetwork_0/subnetwork/dense_1/kernel/RMSProp': [150528, 100], 'adanet/iteration_0/ensemble_t0_cnn_0/weighted_subnetwork_0/subnetwork/dense_2/bias': [10], 'adanet/iteration_0/ensemble_t0_cnn_1/train_subnetwork/adanet/iteration_0/ensemble_t0_